In [44]:
import pandas as pd
import math

In [45]:
ratings = pd.read_csv("dataset/ratings.csv")
n_rows = ratings.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(ratings.head(10).to_string(index=False))

Number of rows: 100836
 userId  movieId  rating  timestamp
      1        1     4.0  964982703
      1        3     4.0  964981247
      1        6     4.0  964982224
      1       47     5.0  964983815
      1       50     5.0  964982931
      1       70     3.0  964982400
      1      101     5.0  964980868
      1      110     4.0  964982176
      1      151     5.0  964984041
      1      157     5.0  964984100


In [57]:
movies = pd.read_csv("dataset/movies.csv")
n_rows = movies.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(movies.head(10).to_string(index=False))

Number of rows: 9742
 movieId                              title                                      genres
       1                   Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
       2                     Jumanji (1995)                  Adventure|Children|Fantasy
       3            Grumpier Old Men (1995)                              Comedy|Romance
       4           Waiting to Exhale (1995)                        Comedy|Drama|Romance
       5 Father of the Bride Part II (1995)                                      Comedy
       6                        Heat (1995)                       Action|Crime|Thriller
       7                     Sabrina (1995)                              Comedy|Romance
       8                Tom and Huck (1995)                          Adventure|Children
       9                Sudden Death (1995)                                      Action
      10                   GoldenEye (1995)                   Action|Adventure|Thriller


In [46]:
def pearsonCorrelation(user1, user2):
    num, den1, den2 = 0.0, 0.0, 0.0
    ratings_user1 = ratings[ratings['userId'] == user1]
    rmean_user1 = ratings_user1['rating'].mean()
    ratings_user2 = ratings[ratings['userId'] == user2]
    rmean_user2 = ratings_user2['rating'].mean()

    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)

    for p in common_items:
        r_1p = (ratings[(ratings['userId'] == user1) & (ratings['movieId'] == p)]['rating'].values)[0]
        r_2p = (ratings[(ratings['userId'] == user2) & (ratings['movieId'] == p)]['rating'].values)[0]
        num += (r_1p - rmean_user1)*(r_2p - rmean_user2)
        den1 += pow((r_1p - rmean_user1), 2)
        den2 += pow((r_2p - rmean_user2), 2)
    den = ((math.sqrt(den1))*(math.sqrt(den2)))
    if den == 0.0:
        return 0.0
    sim = num/den
    return sim

In [47]:
def generatePrediction(user1, item):
    #aggiungi controllo che il film non è stato valutato
    num, den = 0.0, 0.0
    ratings_user1 = ratings[ratings['userId'] == user1]
    if item in ratings_user1['movieId'].tolist():
        return ratings_user1[ratings_user1['movieId'] == item]['rating'].iloc[0]
    rmean_user1 = ratings_user1['rating'].mean()
    # set of users who rated item 
    users = ratings[ratings['movieId'] == item]['userId'].unique()
    for u in users:
        ratings_u = ratings[ratings['userId'] == u]
        rmean_u = ratings_u['rating'].mean()
        r_up = (ratings_u[(ratings_u['movieId'] == item)]['rating'].values)[0]
        similarity = pearsonCorrelation(user1, u)
        num += similarity*(r_up - rmean_u)
        den += abs(similarity)
    if den == 0.0:
        pred = rmean_user1
    else:
        pred = rmean_user1 + (num/den)
    return pred

### (a) For producing group recommendation, we will use the user-based collaborative filtering approach as this implemented in Assignment 1. Specifically, for producing group recommendations, we will first compute the movies recommendations for each user in the group, and then we will aggregate the lists of the individual users, so as to produce a single list of movies for the group. You will implement two well established aggregation methods for producing the group recommendations.

### The first aggregation approach is the <em>average method</em>. The main idea behind this approach is that all members are considered equals. So, the rating of an item for a group of users will be given be averaging the scores of an item across all group members.

In [51]:
def averageMethod(group, item):
    scores_sum = 0.0
    for user in group:
        rating = generatePrediction(user, item)
        scores_sum += rating
    pred = scores_sum / len(group)
    return pred

In [52]:
print(averageMethod([1,4,9,18],50))

4.317527248350769


### The second aggregation method is the <em>least misery method</em>, where one member can act as a veto for the rest of the group. In this case, the rating of an item for a group of users is computed as the minimum score assigned to that item in all group members recommendations.

In [55]:
def leastMiseryMethod(group, item):
    scores = []
    for user in group:
        rating = generatePrediction(user, item)
        scores.append(rating)
    pred = min(scores)
    return pred

In [56]:
print(leastMiseryMethod([1,4,9,18],50))

3.6090081951346242


### Produce a group of 3 users, and for this group, show the top-10 recommendations, i.e., the 10 movies with the highest prediction scores that (i) the average method suggests, and (ii) the least misery method suggest. Use the MovieLens 100K rating dataset.

In [58]:
def topKMovies(group,k):
    all_movie_ids = movies['movieId'].tolist()
    predictions = []
    for m in all_movie_ids:
        print("m: "+str(m))
        pred = averageMethod(group, m)
        predictions.append((m, pred)) 
    sorted_predictions = sorted(predictions, reverse=True, key=lambda x: x[1])
    top_k_predictions = sorted_predictions[:k]
    return top_k_predictions

In [60]:
import concurrent.futures

def topKMoviesPar(group, k):
    all_movie_ids = movies['movieId'].tolist()
    predictions = []
    
    def process_movie(m):
        print("m: " + str(m))
        pred = averageMethod(group, m)
        return (m, pred)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        predictions = list(executor.map(process_movie, all_movie_ids))
    
    sorted_predictions = sorted(predictions, reverse=True, key=lambda x: x[1])
    top_k_predictions = sorted_predictions[:k]
    return top_k_predictions
